# Assignment 7 - Tensorflow
## Alexander Mervar - 3.9.2022

In [1]:
# What version of Python do you have?
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.4.0
Keras Version: 2.4.0

Python 3.8.12 | packaged by conda-forge | (default, Jan 30 2022, 23:33:09) 
[Clang 11.1.0 ]
Pandas 1.4.1
Scikit-Learn 1.0.2
GPU is NOT AVAILABLE


2022-03-08 14:36:00.398754: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set


### Exercise 1
**Using the tensorflow Keras API, build and train a deep network that classifies the MNIST-fashion dataset as accurately as possible. Submit your code to Canvas (10 pts)**

In [2]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

2022-03-08 14:36:01.103051: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-08 14:36:01.334002: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3666 - accuracy: 0.8906
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0994 - accuracy: 0.9699
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0640 - accuracy: 0.9802
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0512 - accuracy: 0.9841
Epoch 5/5
313/313 [==============================] - 0s 1ms/step - loss: 0.0624 - accuracy: 0.9817


[0.06238776072859764, 0.9817000031471252]

### Exercise 2
**Try to optimize your code to achieve a higher percentage accuracy by manipulating parameters such as dropout, stride, and the numbers of units in each layer. Describe in a short essay what network structures and design choices lead to better performance?  Why do you think that is? (10 pts)**

In [16]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(2,2),strides=(1, 1), padding='same', activation='relu', input_shape=(28,28,1))) 
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(2,2),strides=(1, 1), padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

x_train = x_train.reshape(-1,28, 28,1)#Reshape for CNN 
x_test = x_test.reshape(-1,28, 28, 1)

model_log=model.fit(x_train, y_train,
          batch_size=60,
          epochs=10,
          verbose=1,
          validation_split=.3)

model.evaluate(x_test, y_test)

Epoch 1/10
700/700 [==============================] - 31s 44ms/step - loss: 0.7099 - accuracy: 0.7703 - val_loss: 0.1047 - val_accuracy: 0.9681
Epoch 2/10
700/700 [==============================] - 31s 45ms/step - loss: 0.1516 - accuracy: 0.9532 - val_loss: 0.0684 - val_accuracy: 0.9794
Epoch 3/10
700/700 [==============================] - 31s 44ms/step - loss: 0.1032 - accuracy: 0.9679 - val_loss: 0.0564 - val_accuracy: 0.9829
Epoch 4/10
700/700 [==============================] - 30s 43ms/step - loss: 0.0885 - accuracy: 0.9729 - val_loss: 0.0517 - val_accuracy: 0.9842
Epoch 5/10
700/700 [==============================] - 36s 52ms/step - loss: 0.0744 - accuracy: 0.9762 - val_loss: 0.0491 - val_accuracy: 0.9845
Epoch 6/10
700/700 [==============================] - 41s 59ms/step - loss: 0.0690 - accuracy: 0.9782 - val_loss: 0.0452 - val_accuracy: 0.9863
Epoch 7/10
700/700 [==============================] - 35s 51ms/step - loss: 0.0631 - accuracy: 0.9799 - val_loss: 0.0468 - val_accuracy:

[0.03571934998035431, 0.9891999959945679]